In [75]:
#!/usr/bin/env python
# coding: utf-8
# ### Set up master path

path_master = r'C:/Users/Ayushee/Dropbox/Columbia/Spring 2019/Analytics in Practice/Data/Csv/'
# ### Create List with ABR file names to iterate through
import re
import os
import pandas as pd

#read in files' names
# ABR_file_names = list()
# for path, subdirs, files in os.walk(path_master):
#     for filename in files:
#         ABR_file_names.append(filename)

# ### ABR_1 Import
with open(path_master+'April 2017 - Antibiotic by Risk1.csv') as fp:
    df_ABR1 = pd.read_csv(fp,
                          skiprows=[0], 
                          usecols=['Unnamed: 1', 'Treated', 'Repulls', 'Mortalities', 'Treatment Success',
                                   'Case Fatality Rate','$/Hd/Rx', 'Treatment $'])
df_ABR1.rename(columns={'Unnamed: 1': 'Drug', 'Treatment Success':'Treatment Success(%)','Case Fatality Rate':'Case Fatality Rate(%)'}, inplace=True)

# ### ABR_2 Import
# ##### 11 columns, repulls split into 2 columns
with open(path_master+'April 2017 - Antibiotic by Risk2.csv') as fp:
    df_ABR2 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
df_ABR2.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB', '4':'Mortalities', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
# Concatenate Treatment $ column
for i in range(len(df_ABR2)):
    df_ABR2.iloc[i].loc['Treatment $'] = (df_ABR2.iloc[i].loc['Treatment $'])+(df_ABR2.iloc[i].loc['Treatment $B'])
del df_ABR2['Treatment $B']
for i in range(len(df_ABR2)):
    df_ABR2.iloc[i].loc['Repulls'] = (df_ABR2.iloc[i].loc['Repulls'])+(df_ABR2.iloc[i].loc['RepullsB'])
del df_ABR2['RepullsB']

# ### ABR_3 Import & ABR_5 Import ###############################
# ##### 10 columns, mortalities is split into 2 columns
with open(path_master+'April 2017 - Antibiotic by Risk3.csv') as fp:
    df_ABR3 = pd.read_csv(fp, 
                          usecols=['0','1','2','3','4','5','6','8','10'],
                         keep_default_na=False)
df_ABR3.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
# Concatenate mortalities column
for i in range(len(df_ABR3)):
    df_ABR3.iloc[i].loc['Mortalities'] = (df_ABR3.iloc[i].loc['Mortalities'])+(df_ABR3.iloc[i].loc['MortalitiesB'])
del df_ABR3['MortalitiesB']

# ### ABR_4 Import
# ##### 9 columns, same as ABR_1, no separate cleaning needed
with open(path_master+'April 2017 - Antibiotic by Risk4.csv') as fp:
    df_ABR4 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','7','9'], keep_default_na=False)
df_ABR4.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                        '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', '9':'Treatment $',
                        }, inplace=True)

# ### ABR_5 Import
# ##### 10 columns, Mortalities split into 2 columns
with open(path_master+'April 2017 - Antibiotic by Risk5.csv') as fp:
    df_ABR5 = pd.read_csv(fp, usecols=['0','1','2','3','4','5','6','8','10'], keep_default_na=False)
df_ABR5.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
# Concatenate mortalities column
for i in range(len(df_ABR5)):
    df_ABR5.iloc[i].loc['Mortalities'] = (df_ABR5.iloc[i].loc['Mortalities'])+(df_ABR5.iloc[i].loc['MortalitiesB'])
del df_ABR5['MortalitiesB']


# ### Fully merge all ABR versions onto df_ABR
frames = [df_ABR1, df_ABR2, df_ABR3]
df_ABR = pd.concat(frames)

In [78]:

df_ABR5.dtypes

Drug                     object
Treated                  object
Repulls                  object
Mortalities              object
Treatment Success(%)     object
Case Fatality Rate(%)    object
$/Hd/Rx                  object
Treatment $              object
dtype: object

In [70]:
# Remove dashes from columns
for i in range(len(df_ABR)):
    for j in ['Treated','Repulls','Mortalities']:
        if df_ABR.iloc[i].loc[j] == '-':
            df_ABR.iloc[i].loc[j] = 0
        else:
            m = df_ABR.iloc[i].loc[j]
            m = m.replace(' ','')
            df_ABR.iloc[i].loc[j] = m.replace(',','')

# Change str -> int for numerical columns
df_ABR['Treated'] = df_ABR['Treated'].apply(lambda x: int(x))
df_ABR['Repulls'] = df_ABR['Repulls'].apply(lambda x: int(x))
df_ABR['Mortalities'] = df_ABR['Mortalities'].apply(lambda x: int(x))

# Remove dollar signs from columns
df_ABR['Treatment $'] = df_ABR['Treatment $'].apply(lambda x: x[1:])
df_ABR['$/Hd/Rx'] = df_ABR['$/Hd/Rx'].apply(lambda x: x[1:])

# Remove dashes from columns
for i in range(len(df_ABR)):
    for j in ['$/Hd/Rx','Treatment $']:
        if df_ABR.iloc[i].loc[j] == '-':
            df_ABR.iloc[i].loc[j] = 0.0
        else:
            m = df_ABR.iloc[i].loc[j]
            df_ABR.iloc[i].loc[j] = m.replace(',','')
            m = m.replace(' ','')
            df_ABR.iloc[i].loc[j] = m.replace(',','')

# Change str -> float for numerical columns
df_ABR['Treatment $'] = df_ABR['Treatment $'].convert_objects(convert_numeric=True)
df_ABR['$/Hd/Rx'] = df_ABR['$/Hd/Rx'].convert_objects(convert_numeric=True)

# Remove percent signs from columns and convery str -> float
df_ABR['Treatment Success(%)'] = df_ABR['Treatment Success(%)'].apply(lambda x: float(x[:-1]))
df_ABR['Case Fatality Rate(%)'] = df_ABR['Case Fatality Rate(%)'].apply(lambda x: float(x[:-1]))

C:\Users\Ayushee\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Ayushee\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ayushee\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ayushee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A

In [ ]:
# Customised Function to clean fully merged df_ABR columns

def convert_1(x):
    if re.search(pattern = r'\w+',string=x):
        return x
    x=x.strip()
    if x[-1]=='%':
        return float(x.replace('%,'))
    if x=='-':
        return 0
    if x[0]=='$':
        return float(x[1:].replace(',','').replace('-','0'))

def convert_2(x):
    return None

def convert_3(x):
    return None


def ABR_Clean(df):
    """Cleans fully merged df_ABR columns to ensure appropriate dtypes/formats"""
    if len(df.columns)==10:
        for col in df.columns:
            df[col]=df[col].apply(convert_1)
    elif len(df.columns)==11:
        for col in df.columns:
            df[col]=df[col].apply(convert_2)
    else:
        for col in df.columns:
            df[col]=df[col].apply(convert_3)

# ### Run this below to convert to .py file before pushing to GitHub
get_ipython().system('jupyter nbconvert --to script Antibiotics_by_Risk.ipynb')